In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt
import numpy as np
import time
from utils.dataset import VideoDataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

## Dataset

In [ ]:
dataset = VideoDataset(root="../videos/dataset_finales_2020_es/", image_size=(480, 640), sigma=10, transform=ToTensor())
data_loader_train = DataLoader(dataset, batch_size=2)

In [ ]:
frames, labels = dataset[299]
print(frames[0].shape)
plt.imshow(frames[0])
plt.imshow(labels[0], alpha=0.8)
plt.show()